In [1]:
from lib.unrotated_GT import create_circuit as create_circuit_GT
from lib.unrotated_CAT import create_circuit as create_circuit_cat
from lib.Unrotated_Direct import create_circuit as create_circuit_direct
from lib.Rotated_GT import surface_data as Rotated_GT_DATA
from lib.Rotated_GT import generate_circuit as generate_circuit_Rotated_GT
from lib.Rotated_Direct import surface_data as Rotated_Direct_DATA
from lib.Rotated_Direct import generate_circuit  as  generate_circuit_Rotated_Direct
from lib.Rotated_CAT import surface_data as Rotated_CAT_DATA
from lib.Rotated_CAT import generate_circuit as generate_circuit_Rotated_CAT

def Rotated_GT(n, p1, p2,vertical):
    inv= not vertical
    data = Rotated_GT_DATA(n, 2*n,n, 6, display_svg=False,CAT=True,zigzag=True,inverted=inv)
    circuit = generate_circuit_Rotated_GT(data, p1, p2, rounds=n,CAT=True,zigzag=True, a='sw',b='nw',c='se',d='ne',inverted=inv)
    return circuit

def Rotated_Direct(n, p1, p2,vertical):
    inv= not vertical

    data = Rotated_Direct_DATA(n, 2*n,n, 0, display_svg=False,CAT=True,inverted=inv)
    circuit = generate_circuit_Rotated_Direct(data, p1, p2, rounds=n,CAT=True, a='sw',b='nw',c='se',d='ne',inverted=inv)
    return circuit

def Rotated_CAT(n, p1, p2,vertical):
    inv= not vertical
    data =  Rotated_CAT_DATA(n, 2*n,n, 4, display_svg=False,CAT=True,zigzag=True,inverted=inv)
    circuit = generate_circuit_Rotated_CAT(data, p1, p2, rounds=n,CAT=True,zigzag=True, a='sw',b='nw',c='se',d='ne',inverted=inv)
    return circuit


def Unrotated_Direct(n,p1,p2,vertical):
    circuit = create_circuit_direct(2*n,m= n,cut=2*n,num_rounds=n,p1=p1,p2=p2,vertical=vertical,show_grid=False)
    return circuit

def unrotated_CAT(n,p1,p2,vertical):
   # Create the circuit
    circuit = create_circuit_cat(n=2*n,m=n,cut=2*n,num_rounds=n,p1=p1,p2=p2,vertical=vertical,show_grid=False)
    return circuit

def unrotated_GT(n,p1,p2,vertical):
   # Create the circuit
    circuit = create_circuit_GT(n=2*n,m=n,cut=2*n,num_rounds=n,p1=p1,p2=p2,vertical=vertical,show_grid=True)
    return circuit




### Simulation


In [2]:
import sinter
import numpy as np
from typing import List, Dict, Callable, Union, Any 
import pymatching
import math
import csv


def Run_simulation(
    function_name: Callable, 
    multiplier: Union[int, float], 
    vertical: bool,
    results_dict: Dict[str, Dict[str, Dict[Union[int, float], List[Dict[str, Any]]]]]
) -> None:
    base = 3000  

    pr = np.geomspace(0.002, 0.025, num=15)
    n_values = [3, 5, 7, 9, 11]
    
    simulation_results_for_this_configuration: List[Dict[str, Any]] = [] 

    for n in n_values:
        Q3ubit_code_tasks = [
            sinter.Task(
                circuit=function_name(n, p, multiplier * p, vertical),
                json_metadata={'p': p, 'n': n, 'multiplier': multiplier, 'vertical': vertical},
            )
            for p in pr
        ]

        # Sampling
        collected_baconshor_code_stats: List[sinter.TaskStats] = sinter.collect(
            num_workers=9,
            tasks=Q3ubit_code_tasks,
            decoders=['pymatching'],
            max_shots=base * 10000, 
            max_errors=base,       
            print_progress=True,
        )

        for stats in collected_baconshor_code_stats:
            physical_err_rate = stats.json_metadata['p']
            
            num_logical_errors = stats.errors  
            num_shots_conducted = stats.shots   

            logical_err_rate_calculated = 0.0
            if num_shots_conducted > 0:
                logical_err_rate_calculated = num_logical_errors / num_shots_conducted
            elif num_logical_errors > 0: 
                logical_err_rate_calculated = float('nan') 

            simulation_results_for_this_configuration.append({
                'n_distance': n, 
                'physical_error_rate': physical_err_rate,
                'logical_errors_observed': num_logical_errors, 
                'shots_conducted': num_shots_conducted,       
                'logical_error_rate_calculated': logical_err_rate_calculated # P_L
            })

    direction_str = 'Vertical' if vertical else 'Horizontal'
    
    func_name_str = function_name.__name__
    
    if func_name_str not in results_dict:
        results_dict[func_name_str] = {}
    if direction_str not in results_dict[func_name_str]:
        results_dict[func_name_str][direction_str] = {}
    current_multiplier_key = multiplier 
    if current_multiplier_key not in results_dict[func_name_str][direction_str]:
        pass 
    
    results_dict[func_name_str][direction_str][current_multiplier_key] = simulation_results_for_this_configuration

    print(f"Stored results for {func_name_str} | {direction_str} | Multiplier: {current_multiplier_key}")




In [ ]:
import json
from typing import List, Dict, Union, Any 
results_dict: Dict[str, Dict[str, Dict[Union[int, float], List[Dict[str, Any]]]]] = {}

names = [Rotated_Direct, Rotated_CAT, Rotated_GT, Unrotated_Direct, unrotated_CAT, unrotated_GT] 

for name_func in names: 
    for vertical in [True, False]:
        for multiplier in [1, 10]:
            Run_simulation(name_func, multiplier, vertical, results_dict)
            
            print(f"Checkpoint: Saving data after {name_func.__name__}, vertical={vertical}, multiplier={multiplier}")
            with open('test.json', 'w') as f:
                json.dump(results_dict, f, indent=4)


print("All simulation results have been stored successfully in Full-Run.json")
